# Risk Analysis Notebook
Comprehensive risk analysis and drawdown monitoring

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# Risk metrics calculation
def calculate_risk_metrics(returns):
    """Calculate comprehensive risk metrics"""
    metrics = {}
    metrics['annual_return'] = returns.mean() * 252 * 100
    metrics['annual_volatility'] = returns.std() * np.sqrt(252) * 100
    metrics['sharpe_ratio'] = metrics['annual_return'] / metrics['annual_volatility'] if metrics['annual_volatility'] > 0 else 0
    
    # Drawdown metrics
    cumsum = (1 + returns).cumprod()
    running_max = cumsum.expanding().max()
    drawdown = (cumsum - running_max) / running_max
    metrics['max_drawdown'] = drawdown.min() * 100
    
    # Recovery metrics
    metrics['sortino_ratio'] = returns.mean() / returns[returns < 0].std() * np.sqrt(252) if len(returns[returns < 0]) > 0 else 0
    metrics['var_95'] = returns.quantile(0.05) * 100
    metrics['cvar_95'] = returns[returns <= returns.quantile(0.05)].mean() * 100
    
    return metrics

print('Risk metrics functions defined')

In [ ]:
def analyze_drawdown_periods(returns):
    """Analyze drawdown periods and recovery times"""
    cumsum = (1 + returns).cumprod()
    running_max = cumsum.expanding().max()
    drawdown = (cumsum - running_max) / running_max
    
    # Find drawdown periods
    in_dd = drawdown != 0
    dd_periods = []
    start = None
    
    for i, is_dd in enumerate(in_dd):
        if is_dd and start is None:
            start = i
        elif not is_dd and start is not None:
            dd_periods.append({
                'start': start,
                'end': i,
                'duration': i - start,
                'max_dd': drawdown[start:i].min()
            })
            start = None
    
    return pd.DataFrame(dd_periods) if dd_periods else pd.DataFrame()

print('Drawdown analysis functions defined')

In [ ]:
# Risk distribution analysis
def plot_risk_distribution(returns):
    """Plot distribution of returns and risk"""
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Histogram of returns
    axes[0, 0].hist(returns * 100, bins=50, edgecolor='black')
    axes[0, 0].set_xlabel('Daily Return (%)')
    axes[0, 0].set_ylabel('Frequency')
    axes[0, 0].set_title('Distribution of Daily Returns')
    
    # Q-Q plot
    stats.probplot(returns, dist="norm", plot=axes[0, 1])
    axes[0, 1].set_title('Q-Q Plot')
    
    # Cumulative returns
    cumsum = (1 + returns).cumprod()
    axes[1, 0].plot(cumsum.values)
    axes[1, 0].fill_between(range(len(cumsum)), cumsum.values, alpha=0.3)
    axes[1, 0].set_xlabel('Days')
    axes[1, 0].set_ylabel('Cumulative Return')
    axes[1, 0].set_title('Cumulative Returns')
    
    # Drawdown
    running_max = cumsum.expanding().max()
    drawdown = (cumsum - running_max) / running_max * 100
    axes[1, 1].fill_between(range(len(drawdown)), drawdown.values, color='red', alpha=0.3)
    axes[1, 1].set_xlabel('Days')
    axes[1, 1].set_ylabel('Drawdown (%)')
    axes[1, 1].set_title('Underwater Plot')
    
    plt.tight_layout()
    return fig

## Risk Metrics Summary

Key risk indicators for strategy evaluation

In [ ]:
# Example usage (commented out)
# returns = df['strategy_returns']
# metrics = calculate_risk_metrics(returns)
# dd_analysis = analyze_drawdown_periods(returns)
# fig = plot_risk_distribution(returns)
# 
# print('Risk Metrics:')
# for key, value in metrics.items():
#     print(f'{key}: {value:.4f}')

## Advanced Risk Analysis

Rolling Sharpe Ratio and Volatility Analysis